In [16]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier,Pool
import lightgbm as lgb
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
import time

In [2]:
# import Dataset to play with it
train= pd.read_csv("/home/altieris/datascience/data/santander-customer-transaction-prediction/train.csv")
test = pd.read_csv('/home/altieris/datascience/data/santander-customer-transaction-prediction/test.csv')

In [8]:
cols=["target","ID_code"]
X = train.drop(cols,axis=1)
y = train["target"]

In [9]:
X_test  = test.drop("ID_code",axis=1)

In [12]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [14]:
# for get better result chage fold_n to 5
fold_n=5
folds = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=10)

 <a id="6"></a> <br>
# 6- Model Development
So far, we have used two  models, and at this point we add another model and we'll be expanding it soon.
in this section you will see following model:
1. lightgbm
1. RandomForestClassifier
1. DecisionTreeClassifier
1. CatBoostClassifier

## 6-1 lightgbm

In [4]:
# based on following kernel https://www.kaggle.com/dromosys/sctp-working-lgb
params = {'num_leaves': 9,
         'min_data_in_leaf': 42,
         'objective': 'binary',
         'max_depth': 16,
         'learning_rate': 0.0123,
         'boosting': 'gbdt',
         'bagging_freq': 5,
         'bagging_fraction': 0.8,
         'feature_fraction': 0.8201,
         'bagging_seed': 11,
         'reg_alpha': 1.728910519108444,
         'reg_lambda': 4.9847051755586085,
         'random_state': 42,
         'metric': 'auc',
         'verbosity': -1,
         'subsample': 0.81,
         'min_gain_to_split': 0.01077313523861969,
         'min_child_weight': 19.428902804238373,
         'num_threads': 4}

In [17]:
%%time
y_pred_lgb = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    lgb_model = lgb.train(params,train_data,num_boost_round=2000,#change 20 to 2000
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 200)##change 10 to 200
            
    y_pred_lgb += lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)/5

Fold 0 started at Thu Mar  7 23:34:00 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.833564	valid_1's auc: 0.811675
[600]	training's auc: 0.867848	valid_1's auc: 0.84136
[900]	training's auc: 0.885227	valid_1's auc: 0.856314
[1200]	training's auc: 0.896729	valid_1's auc: 0.86598
[1500]	training's auc: 0.904799	valid_1's auc: 0.872299
[1800]	training's auc: 0.911206	valid_1's auc: 0.877422
Did not meet early stopping. Best iteration is:
[2000]	training's auc: 0.914416	valid_1's auc: 0.879595
Fold 1 started at Thu Mar  7 23:35:16 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.831021	valid_1's auc: 0.821954
[600]	training's auc: 0.866447	valid_1's auc: 0.853076
[900]	training's auc: 0.884155	valid_1's auc: 0.867669
[1200]	training's auc: 0.895765	valid_1's auc: 0.876575
[1500]	training's auc: 0.903851	valid_1's auc: 0.882447
[1800]	training's auc: 0.909834	valid_1's auc: 0.886699
Did not meet early s

 <a id="64"></a> <br>
## 6-4 CatBoostClassifier

In [18]:
train_pool = Pool(train_X,train_y)
cat_model = CatBoostClassifier(
                               iterations=3000,# change 25 to 3000 to get best performance 
                               learning_rate=0.03,
                               objective="Logloss",
                               eval_metric='AUC',
                              )
cat_model.fit(train_X,train_y,silent=True)
y_pred_cat = cat_model.predict(X_test)

Now you can change your model and submit the results of other models.

In [19]:
submission_cat = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": y_pred_cat
    })
submission_cat.to_csv('submission_cat.csv', index=False)

In [20]:
submission_lgb = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": y_pred_lgb
    })
submission_lgb.to_csv('submission_lgb.csv', index=False)